# Setup

In [ ]:
# PLEASE READ:
# This notebook here is mostly compiling the results AFTER the benchmarks are
# done. If you use this notebook, please install LISA
#     https://github.com/ARM-software/lisa
# and then workload-processor
#     https://github.com/mrkajetanp/workload-processor.git
# first. After the installation, run this workload via:
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_jankbench.yaml baseline
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_jankbench.yaml test
# This will give you two directories of
#     jankbench_baseline_$RUNS_$DATE
# and
#     jankbench_test_$RUNS_$DATE
# To obtain power consumption results on the two benchmark runs, do
# $ workload-processor process jankbench_baseline_$RUNS_$DATE -m power
# and then
# $ workload-processor process jankbench_test_$RUNS_$DATE -m power
#
# Only after this should you start using this notebook. The reason is that
# these benchmark runs generate a huge amount of traces. Running the benchmarks
# inside this notebook can easily crash Jupyter completely.

In [ ]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

## Runs

In [26]:
jank = WorkloadNotebookAnalysis('/data_sda/honxia02/lisa/', ['jankbench_mainroom_10_1210', 'jankbench_sumroom60_10_1210'])
jank.show()

,test_name,rep,metric,value,unit,iteration,workload,id,wa_path,kernel,tag
0,Bitmap Upload Test,0.0,mean,17.929524,,1,jankbench,wk1,jankbench_mainroom_10_1210,5.18.0-armpower-00029-gc35255a0e739,mainroom
1,Bitmap Upload Test,0.0,std_dev,2.339543,,1,jankbench,wk1,jankbench_mainroom_10_1210,5.18.0-armpower-00029-gc35255a0e739,mainroom
2,Bitmap Upload Test,0.0,count_jank,19.000000,,1,jankbench,wk1,jankbench_mainroom_10_1210,5.18.0-armpower-00029-gc35255a0e739,mainroom
3,Bitmap Upload Test,0.0,jank_p,0.691916,,1,jankbench,wk1,jankbench_mainroom_10_1210,5.18.0-armpower-00029-gc35255a0e739,mainroom
4,Edit Text Input,0.0,mean,6.304410,,1,jankbench,wk1,jankbench_mainroom_10_1210,5.18.0-armpower-00029-gc35255a0e739,mainroom
...,...,...,...,...,...,...,...,...,...,...,...
635,Overdraw Test,0.0,jank_p,0.036643,,10,jankbench,wk1,jankbench_sumroom60_10_1210,5.18.0-armpower-00037-gfdf7806135b3,sumroom60
636,Shadow Grid Fling,0.0,mean,7.305272,,10,jankbench,wk1,jankbench_sumroom60_10_1210,5.18.0-armpower-00037-gfdf7806135b3,sumroom60
637,Shadow Grid Fling,0.0,std_dev,3.306291,,10,jankbench,wk1,jankbench_sumroom60_10_1210,5.18.0-armpower-00037-gfdf7806135b3,sumroom60
638,Shadow Grid Fling,0.0,count_jank,3.000000,,10,jankbench,wk1,jankbench_sumroom60_10_1210,5.18.0-armpower-00037-gfdf7806135b3,sumroom60


benchmark_dirs: ['jankbench_mainroom_10_1210', 'jankbench_sumroom60_10_1210']
tags: ['mainroom', 'sumroom60']
kernels: ['5.18.0-armpower-00029-gc35255a0e739', '5.18.0-armpower-00037-gfdf7806135b3']


# Benchmark scores

In [27]:
jank.plot.jankbench_jank_percentage_line()

:NdOverlay   [tag]
   :Curve   [iteration]   (perc)

In [28]:
jank.plot.jankbench_jank_percentage_bar()

+--------+-----------+-----------------+-------+-----------+
| metric |    tag    |    variable     | value | perc_diff |
+--------+-----------+-----------------+-------+-----------+
| gmean  | mainroom  | jank_percentage |  0.8  |   0.0%    |
| gmean  | sumroom60 | jank_percentage |  0.2  |  -75.44%  |
+--------+-----------+-----------------+-------+-----------+


## Jankbench metrics per iteration

## Bar plot - jank percentage

In [29]:
jank.plot.jankbench_jank_percentage_metric_bar()

+--------+-----------+--------------------------+-------+-----------+
| metric |    tag    |        test_name         | value | perc_diff |
+--------+-----------+--------------------------+-------+-----------+
| jank_p | mainroom  |    Bitmap Upload Test    |  0.3  |   0.0%    |
| jank_p | mainroom  |     Edit Text Input      |  2.5  |   0.0%    |
| jank_p | mainroom  | High-hitrate text render |  1.5  |   0.0%    |
| jank_p | mainroom  |  Image List View Fling   |  0.5  |   0.0%    |
| jank_p | mainroom  |     List View Fling      |  0.6  |   0.0%    |
| jank_p | mainroom  | Low-hitrate text render  |  1.1  |   0.0%    |
| jank_p | mainroom  |      Overdraw Test       |  0.1  |   0.0%    |
| jank_p | mainroom  |    Shadow Grid Fling     |  0.4  |   0.0%    |
| jank_p | sumroom60 |    Bitmap Upload Test    |  0.1  |  -69.9%   |
| jank_p | sumroom60 |     Edit Text Input      |  2.1  |  -16.52%  |
| jank_p | sumroom60 | High-hitrate text render |  0.2  |  -89.15%  |
| jank_p | sumroom60

## Bar plot - mean duration

# Overutilized

## Line plot

In [6]:
jank.plot.overutilized_line()

+--------------+------------+------+------------+------------+
|    metric    |    tag     | time | total_time | percentage |
+--------------+------------+------+------------+------------+
| overutilized | mainfridge | 8.51 |   254.32   |    3.35    |
| overutilized | sumfridge  | 4.37 |   256.08   |    1.71    |
+--------------+------------+------+------------+------------+


# Power usage

## Bar plot

In [30]:
jank.plot.power_meter_bar()

+------------+--------+-----------+-------+-----------+
|  channel   | metric |    tag    | value | perc_diff |
+------------+--------+-----------+-------+-----------+
|    CPU     | gmean  | mainroom  | 146.2 |   0.0%    |
|  CPU-Big   | gmean  | mainroom  | 51.9  |   0.0%    |
| CPU-Little | gmean  | mainroom  | 77.1  |   0.0%    |
|  CPU-Mid   | gmean  | mainroom  | 17.1  |   0.0%    |
|    GPU     | gmean  | mainroom  | 31.7  |   0.0%    |
|   Total    | gmean  | mainroom  | 177.9 |   0.0%    |
|    CPU     | gmean  | sumroom60 | 147.6 |   0.96%   |
|  CPU-Big   | gmean  | sumroom60 | 46.0  |  -11.31%  |
| CPU-Little | gmean  | sumroom60 | 80.2  |   4.05%   |
|  CPU-Mid   | gmean  | sumroom60 | 21.4  |  24.84%   |
|    GPU     | gmean  | sumroom60 | 31.9  |   0.66%   |
|   Total    | gmean  | sumroom60 | 179.5 |   0.9%    |
+------------+--------+-----------+-------+-----------+


In [2]:
main_avg_time = 254.60894279999997
sum60_avg_time = 256.39351809999994
1.0096 * sum60_avg_time / main_avg_time

1.016676370543258